# 소방청 광주지역 검색

In [140]:
import time
from typing import KeysView
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup

In [141]:
import warnings
warnings.filterwarnings('ignore')

In [142]:
driver = webdriver.Chrome() # 여기서 괄호에는 chromedriver.exe 를 생략했다
#구글창을 키는 코드 (크롬드라이브exe파일을이용한다)
# url = 'https://www.google.co.kr/'
# driver = webdriver.Chrome('chromedriver.exe')
# driver.get(url)
# 동적인 데이터를 입력하고 결과를 얻어내기 힘들다
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=35.1493479&myY=126.9218479&searchKeyword='
driver.get(url)

In [143]:
#입력창에 광주를 입력 후 검색 버튼 엔터
driver.find_element(By.ID, 'searchKeyword').send_keys('광주광역시')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [144]:
# 검색 이동 후 뷰티풀습 이용하여 정보 획득
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')
len(lis)
# lis는 페이지 정보

10

In [145]:
#제목
li = lis[0] # 1페이지정보
name = li.select_one('.title').get_text().strip()
name


'광산소방서_광주-평동-119 안전센터'

In [160]:
addr = li.find('address').string.strip()
addr

'광주광역시 서구 화운로 197(화정동)'

In [147]:

tel = li.select_one('.tel').get_text().strip()
tel

'전화공사중'

In [148]:
# 첫페이지크롤링 
lines = []
for li in lis:
    name = li.select_one('.title').get_text().strip()
    addr = li.find('address').string.strip()
    tel = li.select_one('.tel').get_text().strip()
    lines.append([name,addr,tel])
lines

[['광산소방서_광주-평동-119 안전센터', '광주광역시 광산구 월전동 1251', '전화공사중'],
 ['광산소방서_광주-첨단-119 안전센터', '광주광역시 광산구 임방울대로825번길 16(쌍암동)', '062-606-4883'],
 ['광산소방서_광주-신가-119 안전센터', '광주광역시 광산구 수등로 236(신가동)', '062-606-4863'],
 ['광산소방서_광주-비아-119 안전센터', '광주광역시 광산구 비아로12번길 10(비아동)', '062-606-4843'],
 ['광산소방서_광주-월곡-119 안전센터', '광주광역시 광산구 사암로 276(월곡동)', '062-606-4823'],
 ['광산소방서_광주-송정-119 안전센터', '광주광역시 광산구 상무대로 93-1(도산동)', '062-606-4803'],
 ['광산소방서_광주-하남-119 안전센터', '광주광역시 광산구 하남산단1번로 13(하남동)', '062-606-4903'],
 ['북부소방서_광주-동림-119 안전센터', '광주광역시 북구 북문대로 202(동림동)', '062-606-4783'],
 ['북부소방서_광주-두암-119 안전센터', '광주광역시 북구 삼정로 5(두암동)', '062-606-4763'],
 ['북부소방서_광주-일곡-119 안전센터', '광주광역시 북구 설죽로 529(일곡동)', '062-606-4743']]

# 2페이지로이동

In [149]:
# 페이지2 누르는 거에 찍고 카피 xpath
# //*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a

In [150]:
# 페이지2를 클릭
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()

In [151]:
 
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

10

# 다음페이지로이동 

In [152]:
# 다음페이지(3페이지) 이동 성공
driver.find_element(By.CSS_SELECTOR, '.next_page').click()

# 전체

In [153]:
#검색결과건수획득 35건
soup.select('.stations-local-after > p > strong ')[1].get_text() #p랑스트롱은 .클래스의 하위태그
# 스트롱태그 중 두번째니까 1인덱스로 잡아준다
건수 = soup.select('.stations-local-after > p > strong ')[1].get_text()
건수

'29건'

In [154]:
# 건이랑 ''를 지우기
count = int(건수.replace('건','').strip())
count

29

In [155]:
import math
count = math.ceil(count/10)  #ceil실 올림
count

3

In [156]:
driver.get(url)
time.sleep(1)
driver.find_element(By.ID, 'searchKeyword').send_keys('광주광역시')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [157]:
lines=[]
for i in range(count):
    if i >= 1 and i %2==0 : 
        driver.find_element(By.CSS_SELECTOR, '.next_page').click() # 짝수니까 넥스트페이즈로
        time.sleep(2) #이동 후 약간의 시간이 필요하다
    if i >= 1 and i %2==1 : 
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        #이건 홀수니까 두번쨰 페이지로 이동
        time.sleep(2) #이동 후 약간의 시간이 필요하다
         
         
    soup = BeautifulSoup(driver.page_source, 'html.parser') 
    lis = soup.select('.stations-list > li')
    for li in lis:
        name = li.select_one('.title').get_text().strip()
        addr = li.find('address').string.strip()
        tel = li.select_one('.tel').get_text().strip()
        lines.append([name,addr,tel])
len(lines)

29

In [158]:
driver.close() #창닫기

In [159]:
df=pd.DataFrame(lines,columns=['관서명', '주소','전화번호'])
df.tail()

,관서명,주소,전화번호
24,광산소방서,광주광역시 광산구 하남산단1번로 13(하남동),062-613-8813
25,남부소방서,광주광역시 남구 송암로58번길 13(송하동),062-613-8613
26,동부소방서,광주광역시 동구 제봉로 210(대인동),062-613-8413
27,북부소방서,광주광역시 북구 서하로 290(오치동),062-613-8713
28,서부소방서,광주광역시 서구 화운로 197(화정동),062-613-8513
